# RACPIT visualization

![logo](../images/logo128.png) This notebook demonstrates our simulation, preprocessing and visualization pipeline for radar data.
See it rendered
[here](https://htmlpreview.github.io/?https://github.com/fraunhoferhhi/racpit/blob/master/notebooks/visualize.html).

In [ ]:
import os
os.chdir("..")

In [ ]:
from pathlib import Path
import pandas as pd

import numpy as np
import xarray as xr

from utils.synthesize import synthetic_radar
from utils.preprocess import open_recordings, identify_config, raw2rdm

from utils import radar
from utils import skeletons as sk

from utils.visualization import spec_plot, animation_real_synthetic, animation_spec

from ifxaion.daq import Daq

import matplotlib.pyplot as plt
from IPython.display import HTML

from networks.img_transf import MultiTransformNet

import torch
from torch.autograd import Variable

In [ ]:
data_path = Path("/mnt/infineon-radar")

raw_dir = data_path / "daq_x-har"
activity = "5_Walking_Boxing"
path = raw_dir / f"{activity}_converted/recording-2020-01-28_12-37-12"

real_path = data_path / "preprocessed/fixed_size/real"
synth_path = data_path / "preprocessed/fixed_size/synthetic"

itn_config = "D"
itn_recording = "2020-02-05T15:16:08"
itn_path = "models/publication_I.model"

## Simulation

### Radar data loading

In [ ]:
daq = Daq(rec_dir=path)
env = daq.env
recording = daq.radar[2]
rec_config = daq.radar[2].cfg
timestamps = recording.data.index
ts_seconds = timestamps.total_seconds()

In [ ]:
config_name = identify_config(rec_config)

rec_config['RadarName'] = rec_config.pop("Name")
rec_config['cfg'] = config_name
rec_config["activity"] = activity

n_samples = rec_config['SamplesPerChirp']
m_chirps = rec_config['ChirpsPerFrame']

print(f"Synthetizing data for configuration {config_name}")

In [ ]:
frame_interval_ms = np.mean((timestamps[1:] - timestamps[:-1]).total_seconds()) * 1e3
duration_sec = (timestamps[-1] - timestamps[0]).total_seconds()
print(f'Mean frame interval:\t{frame_interval_ms} ms')
print(f'Total duration:\t{duration_sec} seconds')

### Data synthesis

Load skeleton data

In [ ]:
skeletons = sk.load(path, verbose=True)
sk_interp = sk.interpolate(skeletons, timestamps.total_seconds())
sk_da = sk.to_xarray(sk_interp, timestamps, attrs=env)

Synthesize raw data from skeleton points using a radar configuration

In [ ]:
syntheticData = synthetic_radar(skeletons, rec_config, ts_seconds)

assert syntheticData.shape[-2] == m_chirps, "Number of chirps of synthetic data not correct"
assert syntheticData.shape[-1] == n_samples, "Number of samples per chirp of synthetic data not correct"

smin = syntheticData.min()
smax = syntheticData.max()
snorm = (syntheticData - smin) / (smax - smin)
raw_synth = pd.DataFrame({"Timestamps": timestamps, "NormData": [sn for sn in snorm]}).set_index("Timestamps")

The result is a DataFrame

In [ ]:
raw_synth

### Preprocessing

The raw data is processed and converted to an [*x*array](http://xarray.pydata.org/en/stable/) DataArray

In [ ]:
rdm_synth = raw2rdm(raw_synth, rec_config, env, name=f"{activity}-{config_name}")

The Range Doppler Maps can be converted to dB

In [ ]:
rdm_db = xr.apply_ufunc(radar.mag2db, rdm_synth, keep_attrs=True, kwargs={"normalize": True})
rdm_db.assign_attrs(units="dB")

Range & Doppler spectrograms in dB can also be calculated

In [ ]:
rdm_abs = np.abs(rdm_synth)
rspect = rdm_abs.sum(dim="doppler").assign_attrs({"long_name": "Range spectrogram", "units": "dB"})
dspect = rdm_abs.sum(dim="range").assign_attrs({"long_name": "Doppler spectrogram", "units": "dB"})
synth_spects = xr.Dataset({"range_spect": rspect, "doppler_spect": dspect}, attrs=rdm_synth.attrs)
synth_spects = xr.apply_ufunc(radar.mag2db, synth_spects, keep_attrs=True, kwargs={"normalize": True})

synth_spects

### Data animations

Process range & Doppler information from the real recording and extract a short time slice from it

In [ ]:
time_slice = slice("00:00:52", "00:01:07")

skeleton_slice = sk_da.sel(time=time_slice)

rdm_real = raw2rdm(recording.data, rec_config, env, name=f"{activity}-{config_name}")
rdm_rabs = np.abs(rdm_real.sel(time=time_slice))

rng_spect = rdm_rabs.sum(dim="doppler").assign_attrs({"long_name": "Range spectrogram", "units": "dB"})
dopp_spect = rdm_rabs.sum(dim="range").assign_attrs({"long_name": "Doppler spectrogram", "units": "dB"})
real_spects = xr.Dataset({"range_spect": rng_spect, "doppler_spect": dopp_spect}, attrs=rdm_real.attrs)
real_spects = xr.apply_ufunc(radar.mag2db, real_spects, keep_attrs=True, kwargs={"normalize": True})

rdm_real_db = xr.apply_ufunc(radar.mag2db, rdm_rabs, keep_attrs=True, kwargs={"normalize": True})
rdm_synth_db = xr.apply_ufunc(radar.normalize_db, rdm_db.sel(time=time_slice), keep_attrs=True)


In [ ]:
%%capture
anim_rs = animation_real_synthetic(rdm_real_db, rdm_synth_db, skeleton_slice,
                                sensor_loc=rec_config["position"], vmin=-40, notebook=True)

In [ ]:
HTML(anim_rs.to_html5_video())

In [ ]:
%%capture
anim_spects = animation_spec(rdm_real_db, real_spects, vmin=-40, notebook=True)

In [ ]:
HTML(anim_spects.to_html5_video())

## Image transformation

![RACPIT Architecture](../images/model.png)

### Open recordings as a list of `Xarray.Datasets`

The example uses lazy load, but in the GPU `load=True` boosts performance

In [ ]:
real_recs = open_recordings(itn_config, real_path, load=False)
synth_recs = open_recordings(itn_config, synth_path, load=False)

In [ ]:
print(f"{len(real_recs)} recordings have been lazy loaded")
i_rec = [i for i, rec in enumerate(real_recs) if rec.date == itn_recording][0]
print(f"Recording {itn_recording} found at index {i_rec}")

In [ ]:
real_rec = real_recs[i_rec]
synth_rec = synth_recs[i_rec]

Extract short spectrograms from the recordings

In [ ]:
tslice = slice("00:01:39", None)
time_length = 64

real_spects = real_rec.drop_vars('label').sel(time=tslice).isel(time=slice(0,time_length))
synth_spects = synth_rec.drop_vars('label').sel(time=tslice).isel(time=slice(0,time_length))

real_spects = xr.apply_ufunc(radar.normalize_db, real_spects.load(), keep_attrs=True)
synth_spects = xr.apply_ufunc(radar.normalize_db, synth_spects.load(), keep_attrs=True)

### Transform images

Load Image Transformation Network

In [ ]:
dtype = torch.FloatTensor

transformer = MultiTransformNet(num_inputs=2, num_channels=1)
transformer.load_state_dict(torch.load(itn_path))
_ = transformer.eval()

Transform real data with the ITN

In [ ]:
range_inp = torch.from_numpy(real_spects.range_spect.values[None, None, :, :])
dopp_inp = torch.from_numpy(real_spects.doppler_spect.values[None, None, :, :])
spec_input = [Variable(range_inp, requires_grad=False).type(dtype), Variable(dopp_inp, requires_grad=False).type(dtype)]

range_hat, doppler_hat = transformer(spec_input)
range_trans = range_hat.detach().numpy()
doppler_trans = doppler_hat.detach().numpy()

In [ ]:
spectrograms = xr.merge([real_spects.rename_vars(range_spect="range_real", doppler_spect="doppler_real"),
                         synth_spects.rename_vars(range_spect="range_synth", doppler_spect="doppler_synth")],
                        combine_attrs="drop_conflicts")
spectrograms["range_trans"] = (['time', 'range'],  np.squeeze(range_trans), {"units": "dB"})
spectrograms["doppler_trans"] = (['time', 'doppler'],  np.squeeze(doppler_trans), {"units": "dB"})

### Plotting

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

spec_plot(spectrograms[["range_real", "range_trans", "range_synth", "doppler_real", "doppler_trans", "doppler_synth"]],
          axes=axes.flatten(), ax_xlabel=axes[-1], vmin=-40, vmax=0, add_colorbar=False)

im = axes[-1][-1].get_images()[0]
fig.align_ylabels(axes[:,0])
_ = [ax.set_ylabel(None) for ax in axes[:,1:].flatten()]

cbar = plt.colorbar(im, ax=axes, orientation="horizontal")
cbar.set_label("Amplitude [dB]")

for title, ax in zip(("Real data $x$", "Transformed data $\widehat{y}$", "Synthetic data $y$"), axes[0]):
     ax.set_title(title)